In [2]:
import pprint
import os
import sys
import collections

import bioframe
import click
import cooler
import cooltools
import cooltools.expected
import matplotlib.pyplot as plt
import matplotlib.gridspec
from matplotlib.lines import Line2D
import numpy as np
from numpy.lib.function_base import average
from pairlib.scalings import norm_scaling 
import pandas as pd
import pairlib
import pairlib.scalings
import pairtools
from itertools import combinations

from pandas.io.pytables import IndexCol

from itertools import combinations

from pandas.io.pytables import IndexCol

np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

# P(s): contact frequency vs genomic separation

In [ ]:
# path to pairs file
pairs_path = ''
# path to output plot file
out_path = ''
# labels for scaling plot curves
labels = ['']
# assembly name for downloading chromsizes
assembly = ''
# exclude specified chromosomes from scalings
exclude_chroms = ['chrXII', 'chrM']
# path to text file containing centromere start and end positions
centromeres_path = ''
# show average trans contact frequency
show_average_trans = True
# scaling plot title
title = ''
# normalise contact frequency up to 1.0
normalized = True
# plot scaling curves slopes
plot_slope = True

In [7]:
chromsizes = bioframe.fetch_chromsizes(assembly, filter_chroms=False, as_bed=True)
chromsizes = chromsizes[~chromsizes.chrom.isin(exclude_chroms)]
#chromsizes.set_index('chrom', inplace=True)

if centromeres_path:
    centromeres = {}
    with open(centromeres_path) as file:
        for line in file:
            cols = line.split(' ')
            if cols[0] not in exclude_chroms:
                # calculate middle position
                centromeres[cols[0]] = (int(cols[1]) + int(cols[2])) // 2
else:
    centromeres = bioframe.fetch_centromeres(assembly)
    centromeres.set_index('chrom', inplace=True)
    centromeres = centromeres.mid.to_dict()

regions = []

# use chromosomal arms as separate regions if no regions are specified
arms = bioframe.split(chromsizes, centromeres)
regions.append(arms[arms.start < arms.end].reset_index())
# remove 40kb from each side (80kb total) of an arm to remove centromere and telomere regions
arms_trimmed = bioframe.ops.expand(arms, -int(8e4))
# remove arms arms with a length of < 0 after removing side regions
regions.append(arms_trimmed[arms_trimmed.start < arms_trimmed.end].reset_index())

all_scalings = []
all_avg_trans_levels = []

In [8]:
def calc_pair_freqs(scalings, trans_levels, calc_avg_trans, normalized):
    dist_bin_mids = np.sqrt(scalings.min_dist * scalings.max_dist)
    pair_frequencies = scalings.n_pairs / scalings.n_bp2
    mask = pair_frequencies > 0

    avg_trans = None
    if calc_avg_trans:
        avg_trans = (
                trans_levels.n_pairs.astype('float64').sum() /
                trans_levels.np_bp2.astype('float64').sum()
        )

    if normalized:
        norm_fact = pairlib.scalings.norm_scaling_factor(dist_bin_mids, pair_frequencies, anchor=int(1e3))
        pair_frequencies = pair_frequencies / norm_fact
        avg_trans = avg_trans / norm_fact

    return (dist_bin_mids[mask], pair_frequencies[mask]), avg_trans

In [9]:
def plot_scalings(scalings, avg_trans_levels, plot_slope, labels, title, out_path):
    """
    Plot scaling curves from a list of (bin, pair frequencies) tuples.
    """
    fig = plt.figure(figsize=(6, 10))
    gs = matplotlib.gridspec.GridSpec(2, 1, height_ratios=[3, 1.5])
    scale_ax = fig.add_subplot(gs[0, 0])
    slope_ax = fig.add_subplot(gs[1, 0]) if plot_slope else None

    for idx, scalings in enumerate(scalings):
        dist_bin_mids, pair_frequencies = scalings
        scale_ax.loglog(
            dist_bin_mids,
            pair_frequencies,
            label=labels[idx],
            lw=1
        )

        if avg_trans_levels is not None:
            scale_ax.axhline(
                avg_trans_levels[idx],
                ls='dashed',
                c=scale_ax.get_lines()[-1].get_color(),
                lw=1
            )

        if slope_ax is not None:
            slope_ax.semilogx(
                np.sqrt(dist_bin_mids.values[1:] * dist_bin_mids.values[:-1]),
                np.diff(np.log10(pair_frequencies.values)) / np.diff(np.log10(dist_bin_mids.values)),
                label=labels[idx],
                lw=1
            )

    plt.sca(scale_ax)
    plt.grid(lw=0.5,color='gray')
    plt.gca().set_aspect(1.0)
    plt.xlim(1e1, 1e6)
    plt.xlabel('genomic separation (bp)')
    plt.ylabel('contact frequency')

    handles, labels = plt.gca().get_legend_handles_labels()
    if avg_trans_levels is not None:
        handles.append(Line2D([0], [0], color='black', lw=1, ls='dashed'))
        labels.append('average trans')
    plt.legend(handles, labels, loc=(1.025, 0.5), frameon=False)

    if slope_ax is not None:
        plt.sca(slope_ax)
        plt.grid(lw=0.5,color='gray')
        plt.xlim(1e1, 1e6)
        plt.ylim(-3.0, 0.0)
        plt.gca().set_aspect(1.0)
        plt.xlabel('distance (bp)')
        plt.ylabel('log-log slope')

    fig.suptitle(title)
    fig.tight_layout()
    fig.subplots_adjust(top=0.95)

    plt.savefig(out_path, dpi=300)

    plt.close()

In [10]:
for region in regions:
    cis_scalings, trans_levels = pairlib.scalings.compute_scaling(
                    pairs_path,
                    region,
                    chromsizes,
                    dist_range=(int(1e1), int(1e9)),
                    n_dist_bins=128,
                    chunksize=int(1e7)
                )

    cis_scalings = cis_scalings[(cis_scalings.start1 > 0) & (cis_scalings.end1 > 0) & (cis_scalings.start2 > 0) & (cis_scalings.end2 > 0)]

    sc_agg = (cis_scalings
                .groupby(['min_dist', 'max_dist'])
                .agg({'n_pairs': 'sum', 'n_bp2': 'sum'})
                .reset_index()
            )

    cis_scalings, avg_trans = calc_pair_freqs(sc_agg, trans_levels, show_average_trans, normalized)

    all_scalings.append(cis_scalings)
    all_avg_trans_levels.append(avg_trans)

In [11]:
plot_scalings(all_scalings, all_avg_trans_levels, plot_slope, ['whole arms', '80kb-trimmed arms'], 'cdc20td_cdc20', out_path)